In [ ]:
#%pip install ISLP

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [241]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
from ISLP import load_data, confusion_table
from ISLP.models import ModelSpec as MS
import pandas as pd


from sklearn.tree import (DecisionTreeClassifier as DTC,
                          DecisionTreeRegressor as DTR,
                          plot_tree,
                          export_text)
from sklearn.metrics import (accuracy_score,
                             log_loss)
from sklearn.ensemble import \
     (RandomForestRegressor as RF,
      GradientBoostingRegressor as GBR)
from ISLP.bart import BART

In [242]:
# load the training data train.csv
train_data = pd.read_csv('train.csv')
train_data.head()
print(train_data.shape)
print(train_data.columns)
# Remove some features that are supposed to be not useful for prediction

# Access the 'Name' column data
name_column = train_data['Name']

# check how many different names are there in the name column
name_list = name_column.unique().tolist()
print(f"Number of unique names: {len(name_list)}") #207 => too much, ignore name feature

# drop the Name feature
train_data = train_data.drop(columns=['Name'])
print(train_data.shape)

# check how many different locations are there in the Location column
location_column = train_data['Location']
location_list = location_column.unique().tolist()
print(location_list)
print(f"Number of unique locations: {len(location_list)}") 
train_data = train_data.drop(columns=['Location'])# =12, also drop location feature
print(train_data.shape)

# Drop the feature "New-Price" because there is not many data
train_data = train_data.drop(columns=['New_Price'])
print(train_data.shape)



(4470, 16)
Index(['ID', 'Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type',
       'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Colour',
       'Seats', 'No. of Doors', 'New_Price', 'Price'],
      dtype='object')
Number of unique names: 207
(4470, 15)
['Coimbatore', 'Kochi', 'Hyderabad', 'Kolkata', 'Bangalore', 'Delhi', 'Pune', 'Chennai', 'Mumbai', 'Ahmedabad', 'Jaipur', '\\N']
Number of unique locations: 12
(4470, 14)
(4470, 13)


In [243]:
# clean the data
# loop for all columns to remove rows with NaN values
model = MS(train_data.columns, intercept=False)
D = model.fit_transform(train_data)
D_raw = D.copy()
print(D.shape)
nan_index = []
for col in D.columns:
    for i in range(D.shape[0]):
        if D[col][i] == '\\N':
            nan_index.append(i)
nan_index = list(set(nan_index)) # remove duplicates
print(len(nan_index))
# remove the rows that have NaN in any column
for idx in nan_index:
    D = D.drop(index=idx)
print(D.shape)

(4470, 13)
51
(4419, 13)


In [244]:
# Preprocess the year data
D['Year'] = pd.to_numeric(D['Year'])# covert to numeric
# normalize the year data to 0.5 to 1.5
min_year = D['Year'].min()*np.ones(D.shape[0])
max_year = D['Year'].max()*np.ones(D.shape[0])

print(min_year[0], max_year[0])

#normalize to 0.5 to 1.5
D['Year'] = 0.5*np.ones(D.shape[0]) + (D['Year'] - min_year) / (max_year - min_year)
print(D['Year'].min(), D['Year'].max())

print(D['Year'])
print(D.shape)

1998.0 2019.0
0.5 1.5
0       1.214286
1       1.214286
2       0.928571
3       1.166667
4       1.119048
          ...   
4465    1.357143
4466    1.357143
4467    1.309524
4468    1.214286
4469    1.261905
Name: Year, Length: 4419, dtype: float64
(4419, 13)


In [245]:
# Preprocess the Kilometers_Driven data
D['Kilometers_Driven'] = pd.to_numeric(D['Kilometers_Driven'])# covert to numeric
print(D['Kilometers_Driven'].min(), D['Kilometers_Driven'].max())
# normalize the Kilometers_Driven data to 0.5 to 1.5
min_km = D['Kilometers_Driven'].min()*np.ones(D.shape[0])
max_km = D['Kilometers_Driven'].max()*np.ones(D.shape[0])

print(min_km[0], max_km[0])

#normalize to 0.5 to 1.5
D['Kilometers_Driven'] = 0.5*np.ones(D.shape[0]) + (D['Kilometers_Driven'] - min_km) / (max_km - min_km)
print(D['Kilometers_Driven'])
# print(D_raw['Kilometers_Driven'])

171 6500000
171.0 6500000.0
0       0.509072
1       0.512513
2       0.514128
3       0.505089
4       0.509974
          ...   
4465    0.507956
4466    0.504160
4467    0.507974
4468    0.508589
4469    0.507974
Name: Kilometers_Driven, Length: 4419, dtype: float64


In [246]:
# Fuel type preprocessing
fuel_types = D['Fuel_Type'].unique().tolist() # check how many different fuel types
print(fuel_types)
for fuel in fuel_types: # encode each fuel type into a separate binary feature
    D[f'Fuel_Type_{fuel}'] = D['Fuel_Type'].apply(lambda x: 1 if x == fuel else 0)
D = D.drop(columns=['Fuel_Type']) # drop the original Fuel_Type column
print(D.columns) # print to check
# print(D['Fuel_Type_Diesel']) # print to check

['Diesel', 'Petrol', 'CNG', 'LPG']
Index(['ID', 'Year', 'Kilometers_Driven', 'Transmission', 'Owner_Type',
       'Mileage', 'Engine', 'Power', 'Colour', 'Seats', 'No. of Doors',
       'Price', 'Fuel_Type_Diesel', 'Fuel_Type_Petrol', 'Fuel_Type_CNG',
       'Fuel_Type_LPG'],
      dtype='object')


In [247]:
# Process the 'Transmission' column
print(D.shape)
transmission_types = D['Transmission'].unique().tolist() # check how many different transmission types
print(transmission_types)
for transmission in transmission_types: # encode each transmission type into a separate binary feature
    D[f'Transmission_{transmission}'] = D['Transmission'].apply(lambda x: 1 if x == transmission else 0)
D = D.drop(columns=['Transmission']) # drop the original Transmission column
print(D.columns) # print to check
print(D['Transmission_Manual']) # print to check
print(D.shape)

(4419, 16)
['Manual', 'Automatic']
Index(['ID', 'Year', 'Kilometers_Driven', 'Owner_Type', 'Mileage', 'Engine',
       'Power', 'Colour', 'Seats', 'No. of Doors', 'Price', 'Fuel_Type_Diesel',
       'Fuel_Type_Petrol', 'Fuel_Type_CNG', 'Fuel_Type_LPG',
       'Transmission_Manual', 'Transmission_Automatic'],
      dtype='object')
0       1
1       1
2       1
3       1
4       1
       ..
4465    1
4466    1
4467    0
4468    1
4469    0
Name: Transmission_Manual, Length: 4419, dtype: int64
(4419, 17)


In [248]:
#Process the 'Owner_Type' column
owner_types = D['Owner_Type'].unique().tolist() # check how many different owner types
print(owner_types)
for owner in owner_types: # encode each owner type into a separate binary feature
    D[f'Owner_Type_{owner}'] = D['Owner_Type'].apply(lambda x: 1 if x == owner else 0)
D = D.drop(columns=['Owner_Type']) # drop the original Owner_Type column
# print(D.columns) # print to check
# print(D['Owner_Type_Second']) # print to check


['First', 'Second', 'Third', 'Fourth & Above']


In [249]:
print(D['Owner_Type_Second']) # print to check
print(D.shape)

0       0
1       0
2       0
3       0
4       0
       ..
4465    0
4466    0
4467    0
4468    0
4469    0
Name: Owner_Type_Second, Length: 4419, dtype: int64
(4419, 20)


In [250]:
#Preprocess Mileage
print(D['Mileage'])
D['Mileage'] = D['Mileage'].str.split(' ').str[0] # extract the numeric part
D['Mileage'] = pd.to_numeric(D['Mileage'])# covert to numeric
print(D['Mileage'])
# normalize the Mileage data to 0.5 to 1.5
min_mileage = D['Mileage'].min()*np.ones(D.shape[0])
max_mileage = D['Mileage'].max()*np.ones(D.shape[0])
print(min_mileage[0], max_mileage[0])
#normalize to 0.5 to 1.5
D['Mileage'] = 0.5*np.ones(D.shape[0]) + (D['Mileage'] - min_mileage) / (max_mileage - min_mileage)
print(D['Mileage'])

0        17.0 kmpl
1       21.43 kmpl
2        13.8 kmpl
3       21.27 kmpl
4        17.0 kmpl
           ...    
4465     16.0 kmpl
4466     27.3 kmpl
4467     12.7 kmpl
4468     24.7 kmpl
4469     12.0 kmpl
Name: Mileage, Length: 4419, dtype: object
0       17.00
1       21.43
2       13.80
3       21.27
4       17.00
        ...  
4465    16.00
4466    27.30
4467    12.70
4468    24.70
4469    12.00
Name: Mileage, Length: 4419, dtype: float64
0.0 33.54
0       1.006857
1       1.138939
2       0.911449
3       1.134168
4       1.006857
          ...   
4465    0.977042
4466    1.313953
4467    0.878652
4468    1.236434
4469    0.857782
Name: Mileage, Length: 4419, dtype: float64


In [251]:
#Process Engine
print(D['Engine'])
D['Engine'] = D['Engine'].str.split(' ').str[0] # extract the numeric part
D['Engine'] = pd.to_numeric(D['Engine'])# covert to numeric
print(D['Engine'])
# normalize the Engine data to 0.5 to 1.5
min_engine = D['Engine'].min()*np.ones(D.shape[0])
max_engine = D['Engine'].max()*np.ones(D.shape[0])
print(min_engine[0], max_engine[0])
#normalize to 0.5 to 1.5
D['Engine'] = 0.5*np.ones(D.shape[0]) + (D['Engine'] - min_engine) / (max_engine - min_engine)
print(D['Engine'])

0       1405 CC
1       1364 CC
2       1299 CC
3       1396 CC
4       1497 CC
         ...   
4465    2179 CC
4466    1498 CC
4467    2179 CC
4468     796 CC
4469    2987 CC
Name: Engine, Length: 4419, dtype: object
0       1405
1       1364
2       1299
3       1396
4       1497
        ... 
4465    2179
4466    1498
4467    2179
4468     796
4469    2987
Name: Engine, Length: 4419, dtype: int64
624.0 5998.0
0       0.645329
1       0.637700
2       0.625605
3       0.643655
4       0.662449
          ...   
4465    0.789356
4466    0.662635
4467    0.789356
4468    0.532006
4469    0.939710
Name: Engine, Length: 4419, dtype: float64


In [252]:
# Process Power
print(D['Power'])
D['Power'] = D['Power'].str.split(' ').str[0] # extract the numeric part
# there are null values in Power column, we need to handle them
# Check how many null values are there
null_power_idx = []
for i in range(len(D['Power'])):
    try:
        power_value = float(D['Power'].iloc[i])
    except:
        null_power_idx.append(i)
print(null_power_idx)
print(f"Number of null Power values: {len(null_power_idx)}")

# D = D[D['Power'] != 'null'] # remove rows with null Power values
# D['Power'] = pd.to_numeric(D['Power'])# covert to numeric
# print(D['Power'])
# # normalize the Power data to 0.5 to 1.5
# min_power = D['Power'].min()*np.ones(D.shape[0])
# max_power = D['Power'].max()*np.ones(D.shape[0])
# print(min_power[0], max_power[0])
# #normalize to 0.5 to 1.5
# D['Power'] = 0.5*np.ones(D.shape[0]) + (D['Power'] - min_power) / (max_power - min_power)
# print(D['Power'])

0          70 bhp
1        87.2 bhp
2          70 bhp
3       88.76 bhp
4         118 bhp
          ...    
4465      140 bhp
4466     98.6 bhp
4467    187.7 bhp
4468     47.3 bhp
4469      224 bhp
Name: Power, Length: 4419, dtype: object
[25, 164, 221, 230, 643, 662, 681, 758, 784, 825, 898, 906, 927, 942, 1051, 1201, 1339, 1391, 1394, 1467, 1488, 1494, 1555, 1578, 1595, 1618, 1766, 1783, 1791, 1804, 1884, 1885, 1914, 1923, 1986, 2057, 2077, 2104, 2109, 2141, 2333, 2334, 2353, 2362, 2380, 2412, 2455, 2459, 2517, 2562, 2648, 2701, 2804, 2849, 2940, 3027, 3166, 3185, 3217, 3283, 3432, 3450, 3464, 3474, 3483, 3491, 3595, 3622, 3720, 3722, 3798, 3869, 3884, 3903, 3950, 4110, 4133, 4149, 4191, 4206, 4248, 4277, 4395]
Number of null Power values: 83


In [253]:
# remove rows with null Power values
D = D.drop(index=null_power_idx)
print(D.shape)

(4336, 20)


In [254]:
# Process Colour column
color_types = D['Colour'].unique().tolist() # check how many different color types
print(color_types)
for color in color_types: # encode each color type into a separate binary feature
    D[f'Colour_{color}'] = D['Colour'].apply(lambda x: 1 if x == color else 0)
D = D.drop(columns=['Colour']) # drop the original Colour column
print(D.columns) # print to check
print(D.shape)

['Others', 'Black/Silver', 'White']
Index(['ID', 'Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power',
       'Seats', 'No. of Doors', 'Price', 'Fuel_Type_Diesel',
       'Fuel_Type_Petrol', 'Fuel_Type_CNG', 'Fuel_Type_LPG',
       'Transmission_Manual', 'Transmission_Automatic', 'Owner_Type_First',
       'Owner_Type_Second', 'Owner_Type_Third', 'Owner_Type_Fourth & Above',
       'Colour_Others', 'Colour_Black/Silver', 'Colour_White'],
      dtype='object')
(4336, 22)


In [255]:
# Process the Seats column
print(D['Seats'])
D['Seats'] = pd.to_numeric(D['Seats'])# covert to numeric
print(D['Seats'])
# normalize the Seats data to 0.5 to 1.5
min_seats = D['Seats'].min()*np.ones(D.shape[0])
max_seats = D['Seats'].max()*np.ones(D.shape[0])
print(min_seats[0], max_seats[0])
#normalize to 0.5 to 1.5
D['Seats'] = 0.5*np.ones(D.shape[0]) + (D['Seats'] - min_seats) / (max_seats - min_seats)
print(D.shape)
print(D['Seats'])

0       5
1       5
2       5
3       5
4       5
       ..
4465    7
4466    5
4467    5
4468    5
4469    7
Name: Seats, Length: 4336, dtype: object
0       5
1       5
2       5
3       5
4       5
       ..
4465    7
4466    5
4467    5
4468    5
4469    7
Name: Seats, Length: 4336, dtype: int64
2.0 10.0
(4336, 22)
0       0.875
1       0.875
2       0.875
3       0.875
4       0.875
        ...  
4465    1.125
4466    0.875
4467    0.875
4468    0.875
4469    1.125
Name: Seats, Length: 4336, dtype: float64


In [256]:
# Process No. of Doors column
print(D['No. of Doors'])
D['No. of Doors'] = pd.to_numeric(D['No. of Doors'])# covert to numeric
print(D['No. of Doors'])
# normalize the No. of Door data to 0.5 to 1.5
min_doors = D['No. of Doors'].min()*np.ones(D.shape[0])
max_doors = D['No. of Doors'].max()*np.ones(D.shape[0])
print(min_doors[0], max_doors[0])
#normalize to 0.5 to 1.5
D['No. of Doors'] = 0.5*np.ones(D.shape[0]) + (D['No. of Doors'] - min_doors) / (max_doors - min_doors)

print(D.shape)

0       4
1       4
2       4
3       4
4       4
       ..
4465    5
4466    4
4467    4
4468    4
4469    5
Name: No. of Doors, Length: 4336, dtype: object
0       4
1       4
2       4
3       4
4       4
       ..
4465    5
4466    4
4467    4
4468    4
4469    5
Name: No. of Doors, Length: 4336, dtype: int64
2.0 5.0
(4336, 22)


In [257]:
# last, we process the response variable Price
print(D['Price'])
D['Price'] = pd.to_numeric(D['Price'])# covert to numeric
print(D['Price'])
# check if there is any NaN in Price column
nan_price_idx = []
for i in range(len(D['Price'])):
    try:
        price_value = float(D['Price'].iloc[i])
    except:
        nan_price_idx.append(i)
print(nan_price_idx)
print(f"Number of NaN Price values: {len(nan_price_idx)}")
# remove rows with NaN Price values
D = D.drop(index=nan_price_idx)
print(D.shape)

0        2.58
1        6.53
2        1.25
3        3.25
4        5.20
        ...  
4465    12.46
4466     5.85
4467    39.75
4468     2.10
4469    49.00
Name: Price, Length: 4336, dtype: float64
0        2.58
1        6.53
2        1.25
3        3.25
4        5.20
        ...  
4465    12.46
4466     5.85
4467    39.75
4468     2.10
4469    49.00
Name: Price, Length: 4336, dtype: float64
[]
Number of NaN Price values: 0
(4336, 22)


In [261]:
# we save processed D to a csv file
D.to_csv('train_processed.csv', index=False)
